In [1]:
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

In [6]:
import keras

path = keras.utils.get_file('nietzche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

In [8]:
text = open(path).read().lower()
print('말뭉치 크기:', len(text))

말뭉치 크기: 600893


In [9]:
maxlen = 60
step = 3

sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('시퀸스 개수:', len(sentences))
chars = sorted(list(set(text)))
print('고유한 글자:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print('벡터화...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀸스 개수: 200278
고유한 글자: 58
벡터화...


In [10]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

모델의 예측이 주어졌을 때 새로운 글자를 샘플링하는 함수

In [12]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1,60):
    print('에포크', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    
    seed_text = text[start_index: start_index + maxlen]
    print('---- 시드 텍스트:' + seed_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
200278/200278 [==============================] - 160s 800us/step - loss: 1.9848
---- 시드 텍스트:the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the something and the sould of the many and something and something of the conception of the supposition of the good and a domance of the soul of the same to the some of the manking to and in the point of the conscience, and the sould and the sould to a soul of the conception of the sould of the conception of the good and the good and the something to the some of the some with a soul, to the con
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through former of the conscience.--une there soul incase self-case as only from the soul and part and a so the reward and so the world in his farter, the many as a deflect of the belome and
acts of the philosopher falsele, and manking to him

hrvely, ixlack or conscience the bad hand-deluch asign tdes
-aremes and meusuingly accurach.w wold, as obedience-timntratigated forms of higfious
turdicul one understioded, thereby shay dough
god errogation and dallly anif featous of eperiateyaljurs cwrolt
eyphigh opbrout of cagnant fordagion
valpaps
itusulated opposerate--he ; these means. this,
recowiled, 
에포크 5
Epoch 1/1
200278/200278 [==============================] - 157s 783us/step - loss: 1.4578
---- 시드 텍스트:the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the belief the string in the will the spirit that it is the strong and a schopenhauer of the present the present and man be disposes the senses of the consequently, and when the self--the strong and has the strong and which a stands that is the spirit of the still the strong in the same the present the spirit the strong and all the senses of the string that is the great a string the great 

through for the far to been wicl
that but wealityobly obtusituteu-land, innato
the value knouldee
hi? evilsk."--and in church, and
from : anstant be understood. pities, from the simil is
deciinantual dubding vowen, of nato wish this wikene, which unca! bewe
jaymen for a such autyph a failthigius sympathy," promcept,menupance is all bad differentcy so called kbeing, o i sive. by
they be stater, lacs, s
sside
에포크 9
Epoch 1/1
200278/200278 [==============================] - 654s 3ms/step - loss: 1.3936
---- 시드 텍스트:the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the same to the state of the standard of the philosophy and the same that the same to the standard of the same to the same to the same to the same that it is the property of the same to the same that it is also the will to the state of the same that it is the same to man and the such as a standands and the same to the same the same thing in t

the sympathy of its tango and gall, is helf-that seems to telece as--vanity. in facual and anfutcome and the germany a princismy. thu
------ 온도: 1.2
the slowly ascending ranks and classes, in which,
through for could
pezfeet hore has
soliaus. a standress highs, allanrer saintings,
bothers has, you world unexpsedreny sore break untiture -another,
acknowledge
men, to be she cofoped to speciall
becomend
distrust thre,--it is
kjust person of placets and justiunh, modest, perhaps greflicar with undosanese of the geilen younable quscervacious. all ;hrestotible excelyions placed highsse. fest mattatical form 
에포크 13
Epoch 1/1
200278/200278 [==============================] - 655s 3ms/step - loss: 1.3620
---- 시드 텍스트:the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the struggle of the great the cause of the comprehend and the philosophers and proposition to the most soul and propes to the delicate the stren

KeyboardInterrupt: 